# CS246 - Colab 9
## Studying COVID-19

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 217.8MB 60kB/s 
     |████████████████████████████████| 204kB 50.9MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=755785f385c93d601e3b30be9e965ef89e8bd094a330ce059a2fcabd65c07523
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
openjdk-8-jdk-headless is already the newest version (8u242-b08-0ubuntu3~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


Now we authenticate a Google Drive client to download the files we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id='1qgUxXG2rVDPuTvvoamqDQVNh0DDZ4LUh'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_19-covid-Confirmed.csv')

id='1KMR3I0Mz6XHtv5tsjGIIgyLNS0jmniaA'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_19-covid-Deaths.csv')

id='1wKgm-A6p6K79hmtDJKRKuK-Cf1kzsIhw'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_19-covid-Recovered.csv')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [0]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [6]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [7]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-03-10 11:30:36--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.198.214.201, 34.199.212.211, 52.73.183.190, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.198.214.201|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  37.9MB/s    in 0.3s    

2020-03-10 11:30:37 (37.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
http://fecf16b4.ngrok.io


### Data Loading

In this Colab, we will be analyzing the timeseries data of the Coronavirus COVID-19 Global Cases, collected by Johns Hopkins CSSE.

Here you can check a realtime dashboard based on this dataset: [https://www.arcgis.com/apps/opsdashboard/index.html?fbclid=IwAR2hQKsEZ3D38wVtXGryUhP9CG0Z6MYbUM_boPEaV8FBe71wUvDPc65ZG78#/bda7594740fd40299423467b48e9ecf6](https://www.arcgis.com/apps/opsdashboard/index.html?fbclid=IwAR2hQKsEZ3D38wVtXGryUhP9CG0Z6MYbUM_boPEaV8FBe71wUvDPc65ZG78#/bda7594740fd40299423467b48e9ecf6)

---



*   ```confirmed```: dataframe containing the total number of confirmed COVID-19 cases per day, divided by geographical area
*   ```deaths```: dataframe containing the number of deaths per day due to COVID-19, divided by geographical area
*   ```recovered```: dataframe containing the number of recoevered patients per day, divided by geographical area







In [0]:
confirmed = spark.read.csv('time_series_19-covid-Confirmed.csv', header=True)
deaths = spark.read.csv('time_series_19-covid-Deaths.csv', header=True)
recovered = spark.read.csv('time_series_19-covid-Recovered.csv', header=True)

In [9]:
confirmed.printSchema()

root
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: string (nullable = true)
 |-- Long: string (nullable = true)
 |-- 1/22/20: string (nullable = true)
 |-- 1/23/20: string (nullable = true)
 |-- 1/24/20: string (nullable = true)
 |-- 1/25/20: string (nullable = true)
 |-- 1/26/20: string (nullable = true)
 |-- 1/27/20: string (nullable = true)
 |-- 1/28/20: string (nullable = true)
 |-- 1/29/20: string (nullable = true)
 |-- 1/30/20: string (nullable = true)
 |-- 1/31/20: string (nullable = true)
 |-- 2/1/20: string (nullable = true)
 |-- 2/2/20: string (nullable = true)
 |-- 2/3/20: string (nullable = true)
 |-- 2/4/20: string (nullable = true)
 |-- 2/5/20: string (nullable = true)
 |-- 2/6/20: string (nullable = true)
 |-- 2/7/20: string (nullable = true)
 |-- 2/8/20: string (nullable = true)
 |-- 2/9/20: string (nullable = true)
 |-- 2/10/20: string (nullable = true)
 |-- 2/11/20: string (nullable = true)
 |-- 2/12/20: string (

### Your Task

We are aware of the great deal of stress we are all experiencing because of the spread of the Coronavirus. As such, we decided to conclude our series of Colabs with a **lightweight task** -- given everything you have learned about Spark during the quarter, this Colab should take you just a few minutes to complete.

At the same time, we turned this into an opportunity to raise awareness about the extent of the COVID-19 epidemic.

[Stay healthy, wash your hands often](https://www.cdc.gov/coronavirus/2019-ncov/about/index.html), and invest the time you saved on this Colab to be emotionally supportive to your friends and family!

Signed, *the CS246 teaching staff*

In [12]:
# YOUR CODE HERE
def md5(s):
  import hashlib
  m = hashlib.md5()
  m.update(s.encode())
  return m.hexdigest()[:2]

md5('3/5/20')

'da'

In [47]:
confirmed.show()

+--------------+--------------+------------------+------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+
|Province/State|Country/Region|               Lat|              Long|1/22/20|1/23/20|1/24/20|1/25/20|1/26/20|1/27/20|1/28/20|1/29/20|1/30/20|1/31/20|2/1/20|2/2/20|2/3/20|2/4/20|2/5/20|2/6/20|2/7/20|2/8/20|2/9/20|2/10/20|2/11/20|2/12/20|2/13/20|2/14/20|2/15/20|2/16/20|2/17/20|2/18/20|2/19/20|2/20/20|2/21/20|2/22/20|2/23/20|2/24/20|2/25/20|2/26/20|2/27/20|2/28/20|2/29/20|3/1/20|3/2/20|3/3/20|3/4/20|3/5/20|
+--------------+--------------+------------------+------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+

Consider only the most recent data points in the timeseries, and compute:


*   number of confirmed COVID-19 cases across the globe
*   number of deaths due to COVID-19 (across the globe)
*   number of recovered patients across the globe



In [80]:
# YOUR CODE HERE
def list_sum(l):
  sum = 0
  for x in l:
    sum += int(x)
  return sum
confirmed_list = confirmed.select('3/5/20').rdd.flatMap(lambda x: x).collect()
deaths_list = deaths.select('3/5/20').rdd.flatMap(lambda x: x).collect()
recovered_list = recovered.select('3/5/20').rdd.flatMap(lambda x: x).collect()

print("number of confirmed COVID-19 cases across the globe is: {}".format(list_sum(confirmed_list)))
print("number of deaths due to COVID-19 cases across the globe is: {}".format(list_sum(deaths_list)))
print("number of recovered patients across the globe is: {}".format(list_sum(recovered_list)))

number of confirmed COVID-19 cases across the globe is: 97886
number of deaths due to COVID-19 cases across the globe is: 3348
number of recovered patients across the globe is: 53797


Consider only the most recent data points in the timeseries, and filter out the geographical locations where less than 50 cases have been confirmed.
For the areas still taken into consideration after the filtering step, compute the ratio between number of recovered patients and number of confirmed cases.

In [0]:
# YOUR CODE HERE
confirmed_list = confirmed.select('Province/State', 'Country/Region', col('3/5/20').alias('confirmed'))
deaths_list = deaths.select('Province/State', 'Country/Region', col('3/5/20').alias('deaths'))
recovered_list = recovered.select('Province/State', 'Country/Region', col('3/5/20').alias('recovered'))
result = confirmed_list.join(deaths_list, ['Province/State', 'Country/Region'])
result = result.join(recovered_list, ['Province/State', 'Country/Region'])
result = result.withColumn("%recovered", result.recovered / result.confirmed)
result = result.withColumn("%deaths", result.deaths / result.confirmed)

Following the same filtering strategy as above, now compute the ratio between number of deaths and number of confirmed cases.

In [82]:
result = result.filter(result.confirmed >= 50).sort("%recovered", ascending = False)
result.show()

+--------------+--------------+---------+------+---------+------------------+--------------------+
|Province/State|Country/Region|confirmed|deaths|recovered|        %recovered|             %deaths|
+--------------+--------------+---------+------+---------+------------------+--------------------+
|         Anhui|Mainland China|      990|     6|      970|0.9797979797979798|0.006060606060606061|
|         Henan|Mainland China|     1272|    22|     1239|0.9740566037735849| 0.01729559748427673|
|        Yunnan|Mainland China|      174|     2|      169|0.9712643678160919|0.011494252873563218|
|       Jiangxi|Mainland China|      935|     1|      901|0.9636363636363636|0.001069518716577...|
|         Hebei|Mainland China|      318|     6|      304|0.9559748427672956|0.018867924528301886|
|        Shanxi|Mainland China|      133|     0|      126|0.9473684210526315|                 0.0|
|         Jilin|Mainland China|       93|     1|       88| 0.946236559139785|0.010752688172043012|
|       Ti

In [83]:
result = result.filter(result.confirmed >= 50).sort("%deaths", ascending = False)
result.show()

+--------------------+--------------+---------+------+---------+--------------------+--------------------+
|      Province/State|Country/Region|confirmed|deaths|recovered|          %recovered|             %deaths|
+--------------------+--------------+---------+------+---------+--------------------+--------------------+
|     King County, WA|            US|       51|    10|        1|  0.0196078431372549| 0.19607843137254902|
|               Hubei|Mainland China|    67466|  2902|    40592|  0.6016660243678297| 0.04301425903418018|
|            Xinjiang|Mainland China|       76|     3|       70|  0.9210526315789473|0.039473684210526314|
|              Hainan|Mainland China|      168|     6|      158|  0.9404761904761905| 0.03571428571428571|
|        Heilongjiang|Mainland China|      481|    13|      379|  0.7879417879417879| 0.02702702702702703|
|             Tianjin|Mainland China|      136|     3|      128|  0.9411764705882353|0.022058823529411766|
|               Gansu|Mainland China|

Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**!